# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bandarbeyla,9.4942,50.8122,24.38,79,44,14.76,SO,1689698788
1,1,mazamari,-11.3258,-74.5308,24.21,85,91,0.72,PE,1689698788
2,2,college,64.8569,-147.8028,16.56,85,75,1.54,US,1689698699
3,3,island harbour,50.7240,-1.2758,22.86,54,52,1.79,GB,1689698789
4,4,laguna,38.4210,-121.4238,31.16,48,0,5.14,US,1689698789


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 29 degrees but higher than 23
# Cloudiness less than 4
# Wind speed less than 3.0 m/s

weather_filtered = city_data_df.loc[(city_data_df["Max Temp"] < 29)
                                       & (city_data_df["Max Temp"] > 23) 
                                       & (city_data_df["Cloudiness"] < 4)
                                       & (city_data_df["Wind Speed"] < 3.0)]

# Drop any rows with null values
weather_filtered.dropna()

# Display sample data
weather_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
159,159,ludwigsfelde,52.3000,13.2667,26.02,41,0,1.54,DE,1689698817
215,215,jinchang,38.4953,102.1739,26.21,28,1,0.75,CN,1689698829
520,520,changping,40.2186,116.2197,23.80,52,0,1.70,CN,1689698908


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
159,ludwigsfelde,DE,52.3000,13.2667,41,
215,jinchang,CN,38.4953,102.1739,28,
520,changping,CN,40.2186,116.2197,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ludwigsfelde - nearest hotel: Hotel Ambassador Ludwigsfelde
jinchang - nearest hotel: No hotel found
changping - nearest hotel: 速8酒店


,City,Country,Lat,Lng,Humidity,Hotel Name
159,ludwigsfelde,DE,52.3000,13.2667,41,Hotel Ambassador Ludwigsfelde
215,jinchang,CN,38.4953,102.1739,28,No hotel found
520,changping,CN,40.2186,116.2197,52,速8酒店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)